In [2]:
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community import modularity
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import os


In [71]:


directory = '/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/projection_parasites/resource_allocation_naive.csv'

In [80]:
df = pd.read_csv(directory)
df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)

In [82]:
# Create the graph with weights
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight')

Merging two dataframes:
- we use locality from nodes.csv and edges.csv
- groups we take from edges.csv and nodes.csv

First Part: extract most frequent localities from nodes and edges .csv

In [ ]:
nodes_csv = pd.read_csv('../data/nodes.csv')
edges_csv = pd.read_csv('../data/edges.csv')
nodes_csv.head()

In [ ]:

edges_csv.head()

In [8]:
import sys
parasites_df = nodes_csv[nodes_csv[" is_host"] == 0].copy()

sys.path.append(os.path.join(os.getcwd(), '../', 'data'))
from common_localities import locality_to_common_locality
most_frequent_localities = []
for id_parasite, parasite in parasites_df.iterrows():
    animals_idx = list(edges_csv[edges_csv[" target"] == id_parasite]["# source"]) + list(
        edges_csv[edges_csv["# source"] == id_parasite][" target"]
    )
    most_frequent_locality = nodes_csv.loc[animals_idx][' locality'].mode().loc[0]
    most_frequent_locality = locality_to_common_locality[most_frequent_locality]
    most_frequent_localities.append(most_frequent_locality)

 

In [9]:
parasites_df.drop(columns=[' full_name', ' locality', ' group', ' is_host', ' _pos'], inplace=True)
parasites_df.rename(columns={"# index": "nodes_index"}, inplace=True)

In [ ]:
parasites_df.head()

In [ ]:
parasites_df['most_frequent_locality'] = most_frequent_localities
parasites_df

Second part: extract correct groups of parasties and most frequent groups of animals species from original data

In [ ]:
original_data = pd.read_csv('../data/helminths.csv', encoding='ISO-8859-1').sort_values(by='Host')
original_data.head()

In [222]:
def random_mode(series: pd.Series) -> pd.Series:
    # could be NaN, then just return it
    if series.isnull().all():
        return []
    return series.mode().sample()

In [ ]:
group_hostgroup_original_data= original_data.groupby(by=['Parasite', 'group', 'Host', 'hostgroup'], dropna=False).size().reset_index().drop(columns=[0])
# take the most frequent hostgroup for each parasite
group_hostgroup_original_data= group_hostgroup_original_data.groupby(by=['Parasite', 'group'])['hostgroup'].agg(random_mode).to_frame().reset_index()
group_hostgroup_original_data 

In [ ]:
group_hostgroup_original_data.head()

In [ ]:
parasites_df

In [ ]:
# Merge the DataFrames based on the conditions
merged_df = parasites_df.merge(
    group_hostgroup_original_data,
    left_on=[' name'],  # Columns in attribute_df
    right_on=['Parasite'],  # Columns in extra_attributes
    how='inner',  # Use 'inner' to keep only matching rows
)

# Display the merged DataFrame
merged_df.drop(columns=['Parasite'], inplace=True)
merged_df.head()

In [225]:
merged_df.to_csv('../data/final_metadata.csv')

In [ ]:

for i in range(len(files)):
    # Read the CSV file
    df = pd.read_csv('/Users/sunechristiansen/sune/network_analysis/project/parasites_network_analysis/projections_with_backbonings/' + files[i])

    # Rename columns
    df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)

    # Create the graph with weights
    G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight')

    print(f'name: {files[i]}, edges: {len(G.edges(data=True))}, nodes: {len(G.nodes)}')


In [2]:
resource_all_df = pd.read_csv('resource_allocation_naive.csv')
# Rename columns
resource_all_df.rename(columns={'src': 'source', 'trg': 'target', 'nij': 'weight'}, inplace=True)

# Create the graph with weights
G = nx.from_pandas_edgelist(resource_all_df, source='source', target='target', edge_attr='weight')

In [ ]:


""" # Find all connected components
connected_components = list(nx.connected_components(filtered_G))

# Find the largest connected component
largest_component = max(connected_components, key=len)

# Create a subgraph of the largest connected component
largest_subgraph = filtered_G.subgraph(largest_component) """

# Print details
print(f"Number of nodes in largest component: {len(largest_subgraph.nodes)}")
print(f"Number of edges in largest component: {len(largest_subgraph.edges)}")


In [ ]:
# Brute force find highest modularity resource allocation plus threshold:
for i in range(100):
    threshold = i/100
    # Get all edge weights
    edge_weights = nx.get_edge_attributes(G, 'weight')

    # Filter edges with weights above the threshold
    filtered_edges = [(u, v, w) for (u, v), w in edge_weights.items() if w >= threshold]

    # Create a new graph with the filtered edges
    filtered_G = nx.Graph()  # Use nx.DiGraph() if the original graph is directed
    filtered_G.add_weighted_edges_from(filtered_edges)

    communities = greedy_modularity_communities(filtered_G, weight=None)

    modularity_r = modularity(filtered_G, communities)
    print("threshold: ", threshold, " & modularity: ", modularity_r)
    